## **Visualización de Comunicación Segura con Cifrado RSA + Vigenère en una Red Geográfica**



Importación de librerías.

Definimos el alfabeto que se utilizará para el cifrado Vigenère.

Coordenadas geográficas de las ciudades colombianas para la visualización.

In [52]:
# CIFRADO VIGENÈRE EXTENDIDO CON RED SIMULADA

import networkx as nx
 # Importa la librería networkx para trabajar con grafos (redes)
print("Librería networkx importada para trabajar con grafos.")
import string
 # Importa la librería string para manejar cadenas de texto y alfabetos
print("Librería string importada para manejar cadenas de texto y alfabetos.")
import random
 # Importa la librería random para generar números aleatorios y selecciones al azar
print("Librería random importada para generar números aleatorios.")
import hashlib
 # Importa la librería hashlib para funciones de hashing seguras como SHA256
print("Librería hashlib importada para funciones de hashing.")
from Crypto.PublicKey import RSA
 # Importa la clave pública de RSA de la librería PyCryptodome para cifrado asimétrico
print("Clave pública RSA importada de PyCryptodome.")
from Crypto.Cipher import PKCS1_OAEP
 # Importa el cifrado PKCS1_OAEP de PyCryptodome para cifrado RSA seguro
print("Cifrado PKCS1_OAEP importado de PyCryptodome.")
import binascii
 # Importa la librería binascii para conversiones entre binario y ASCII binario
print("Librería binascii importada para conversiones binarias.")
import plotly.graph_objects as go
 # Importa graph_objects de plotly para crear gráficos interactivos
print("Plotly graph_objects importado para gráficos interactivos.")


# Definimos el alfabeto que se utilizará para el cifrado Vigenère
ALFABETO = string.ascii_letters + string.digits + string.punctuation + ' '
print(f"\nAlfabeto definido para cifrado Vigenère: '{ALFABETO}'")

# Coordenadas geográficas de las ciudades colombianas para la visualización
CIUDADES_COORDS = {
    "Bogotá": (4.711, -74.0721),
    "Medellín": (6.2442, -75.5812),
    "Cali": (3.4516, -76.5319),
    "Barranquilla": (10.9685, -74.7813),
    "Cartagena": (10.3910, -75.4794),
    "Bucaramanga": (7.1193, -73.1227),
    "Cúcuta": (7.8939, -72.5078),
    "Pereira": (4.8087, -75.6906),
    "Manizales": (5.0689, -75.5174),
    "Ibagué": (4.4389, -75.2322),
    "Pasto": (1.2136, -77.2811),
    "Neiva": (2.9965, -75.2908),
    "Villavicencio": (4.1438, -73.6224),
    "Santa Marta": (11.2408, -74.2048),
    "Montería": (8.7609, -75.8822),
    "Armenia": (4.5388, -75.6816),
    "Sincelejo": (9.3047, -75.3973),
    "Popayán": (2.4410, -76.6029),
    "Tunja": (5.5350, -73.3677),
    "Riohacha": (11.5444, -72.9078),
    "Valledupar": (10.4634, -73.2934),
    "Apartadó": (7.8834, -76.6117),
    "Florencia": (1.6137, -75.6060),
    "Quibdó": (5.6932, -76.6585),
    "Arauca": (7.0841, -70.7611),
    "San Andrés": (12.5833, -81.7000),
    "Leticia": (-4.2167, -69.9333)
}
print(f"Coordenadas de {len(CIUDADES_COORDS)} ciudades colombianas cargadas.")

Librería networkx importada para trabajar con grafos.
Librería string importada para manejar cadenas de texto y alfabetos.
Librería random importada para generar números aleatorios.
Librería hashlib importada para funciones de hashing.
Clave pública RSA importada de PyCryptodome.
Cifrado PKCS1_OAEP importado de PyCryptodome.
Librería binascii importada para conversiones binarias.
Plotly graph_objects importado para gráficos interactivos.

Alfabeto definido para cifrado Vigenère: 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ '
Coordenadas de 27 ciudades colombianas cargadas.


***Esta función se encarga de cifrar un mensaje de texto utilizando el método de cifrado Vigenère.***

In [53]:
def vigenere_cifrar(mensaje, clave):
    """
    Cifra un mensaje usando el cifrado Vigenère con una clave dada.

    Toma un mensaje y una clave como entrada. Itera sobre cada carácter del mensaje y,
    si el carácter se encuentra en el ALFABETO definido, lo cifra utilizando la clave
    y la posición del carácter en el alfabeto. Los caracteres que no están en el
    alfabeto se dejan sin cifrar.

    Args:
        mensaje (str): El mensaje a cifrar.
        clave (str): La clave para el cifrado Vigenère.

    Returns:
        str: El mensaje cifrado.
    """
    print(f"\nIniciando cifrado Vigenère para el mensaje: '{mensaje}' con clave: '{clave}'")
    # Itera sobre cada carácter del mensaje con su índice
    cifrado = ''.join(
        # Cifra el carácter si está en el alfabeto
        ALFABETO[(ALFABETO.index(c) + ALFABETO.index(clave[i % len(clave)])) % len(ALFABETO)]
        if c in ALFABETO else c # Deja el carácter sin cifrar si no está en el alfabeto
        for i, c in enumerate(mensaje)
    )
    print(f"Mensaje cifrado: '{cifrado}'")
    return cifrado

def vigenere_descifrar(mensaje, clave):
    """
    Descifra un mensaje usando el cifrado Vigenère con una clave dada.

    Toma el mensaje cifrado y la misma clave utilizada para cifrarlo. Itera sobre
    cada carácter del mensaje cifrado y, si está en el ALFABETO, lo descifra
    utilizando la clave de forma inversa al cifrado. Los caracteres no presentes
    en el alfabeto se mantienen igual.

    Args:
        mensaje (str): El mensaje a descifrar.
        clave (str): La clave utilizada para el cifrado Vigenère.

    Returns:
        str: El mensaje descifrado.
    """
    print(f"\nIniciando descifrado Vigenère para el mensaje: '{mensaje}' con clave: '{clave}'")
    # Itera sobre cada carácter del mensaje cifrado con su índice
    descifrado = ''.join(
        # Descifra el carácter si está en el alfabeto
        ALFABETO[(ALFABETO.index(c) - ALFABETO.index(clave[i % len(clave)])) % len(ALFABETO)]
        if c in ALFABETO else c # Deja el carácter sin descifrar si no está en el alfabeto
        for i, c in enumerate(mensaje)
    )
    print(f"Mensaje descifrado: '{descifrado}'")
    return descifrado

print("\nFunciones vigenere_cifrar y vigenere_descifrar definidas.")


Funciones vigenere_cifrar y vigenere_descifrar definidas.


Esta función se encarga de verificar si un mensaje que supuestamente ha sido "firmado"


(usando la función firmar_mensaje que vimos antes)

realmente no ha sido modificado desde que se le adjuntó la "firma".

In [54]:
def firmar_mensaje(mensaje):
    """
    Firma digitalmente un mensaje calculando su hash SHA256 y adjuntando los primeros 8 caracteres.

    Simula una firma digital simple. Toma un mensaje, calcula su hash SHA256 y luego
    toma los primeros 8 caracteres de ese hash. Devuelve el mensaje original
    concatenado con "||" y estos 8 caracteres del hash como la "firma". Esto permite
    verificar si el mensaje fue alterado posteriormente (aunque no proporciona
    autenticación real como una firma digital con claves privadas/públicas).

    Args:
        mensaje (str): El mensaje a firmar.
    Returns:
        str: El mensaje firmado en formato "mensaje||firma".
    """
    print(f"\nFirmando mensaje: '{mensaje}'")
    # Calcula el hash SHA256 del mensaje
    hash_completo = hashlib.sha256(mensaje.encode()).hexdigest()
    print(f"Hash SHA256 completo: {hash_completo}")
    # Toma los primeros 8 caracteres del hash
    hash8 = hash_completo[:8]
    print(f"Primeros 8 caracteres del hash (firma simulada): {hash8}")
    # Concatena el mensaje original con "||" y los 8 caracteres del hash
    mensaje_firmado = f"{mensaje}||{hash8}"
    print(f"Mensaje firmado: '{mensaje_firmado}'")
    return mensaje_firmado

def verificar_firma(mensaje_firmado):
    """
    Verifica la firma digital de un mensaje firmado.

    Verifica la "firma" de un mensaje que fue firmado previamente con firmar_mensaje.
    Toma el mensaje_firmado en formato "mensaje||firma", lo divide en el mensaje
    original y la firma. Recalcula el hash SHA256 del mensaje original extraído y
    compara los primeros 8 caracteres con la firma proporcionada.

    Args:
        mensaje_firmado (str): El mensaje firmado en formato "mensaje||firma".

    Returns:
        tuple: Una tupla que contiene un booleano (True si la firma es válida, False si no)
               y el mensaje original extraído.
    """
    print(f"\nVerificando firma para el mensaje firmado: '{mensaje_firmado}'")
    try:
        # Divide el mensaje firmado en mensaje original y firma
        mensaje, firma = mensaje_firmado.rsplit('||', 1)
        print(f"Mensaje extraído: '{mensaje}', Firma extraída: '{firma}'")
        # Recalcula el hash del mensaje original extraído y compara los primeros 8 caracteres con la firma
        hash_recalculado = hashlib.sha256(mensaje.encode()).hexdigest()[:8]
        print(f"Hash recalculado del mensaje extraído: {hash_recalculado}")
        valido = firma == hash_recalculado
        print(f"Comparando firma extraída ({firma}) con hash recalculado ({hash_recalculado}). Válido: {valido}")
        return valido, mensaje
    except ValueError:
        # Si no se puede dividir el mensaje (formato incorrecto), la firma es inválida
        print("Error: Formato de mensaje firmado incorrecto. No se encontró '||'.")
        return False, mensaje_firmado

print("\nFunciones firmar_mensaje y verificar_firma definidas.")


Funciones firmar_mensaje y verificar_firma definidas.


**(crear_red_dispositivos)**Esta función se encarga de construir la red simulada de dispositivos (nodos) que se utilizará en el programa. Piensa en ella como la creadora del mapa y los puntos de conexión.

In [55]:
def crear_red_dispositivos(n):
    """
    Crea una red aleatoria de dispositivos (nodos) distribuidos en ciudades colombianas.

    A cada nodo se le asigna un par de claves RSA (pública y privada) y una ubicación
    geográfica. Crea un grafo aleatorio usando el modelo de Erdos-Renyi y asegura
    que sea conexo (que todos los nodos estén conectados).

    Args:
        n (int): El número de nodos (dispositivos) a crear.

    Returns:
        nx.Graph: El grafo que representa la red de dispositivos.
    """
    print(f"\nCreando una red aleatoria de {n} dispositivos.")
    # Obtiene la lista de ciudades y la mezcla aleatoriamente
    ciudades = list(CIUDADES_COORDS.keys())
    random.shuffle(ciudades)
    print(f"Ciudades disponibles mezcladas para asignación: {ciudades[:5]}...")

    # Crea un grafo aleatorio usando el modelo de Erdos-Renyi
    G = nx.erdos_renyi_graph(n, 0.4)
    print(f"Grafo inicial creado con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")
    # Asegura que el grafo sea conexo (todos los nodos conectados)
    while not nx.is_connected(G):
        print("Grafo no conexo. Regenerando...")
        G = nx.erdos_renyi_graph(n, 0.4)
        print(f"Grafo regenerado con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")
    print("Grafo conexo creado exitosamente.")

    # Asigna claves RSA y ubicación a cada nodo
    print("Asignando claves RSA y ubicación a cada nodo...")
    for nodo in G.nodes:
        # Genera un par de claves RSA (pública y privada)
        key = RSA.generate(2048)
        G.nodes[nodo]['rsa_private'] = key
        G.nodes[nodo]['rsa_public'] = key.publickey()
        # Asigna una ciudad al nodo
        ciudad = ciudades[nodo % len(ciudades)]
        G.nodes[nodo]['ciudad'] = ciudad
        G.nodes[nodo]['pos'] = CIUDADES_COORDS[ciudad] # Asigna coordenadas de la ciudad
        # print(f" - Nodo {nodo}: Claves RSA generadas, Ciudad asignada: {ciudad}") # Descomentar para ver asignaciones individuales

    print("Claves RSA y ubicaciones asignadas a todos los nodos.")
    return G

print("\nFunción crear_red_dispositivos definida.")


Función crear_red_dispositivos definida.


Ahora veamos la función (**enviar_mensaje_unico,**) que es el corazón de la
simulación, ya que modela cómo un mensaje viaja a través de la red, cómo se cifra y descifra, y cómo un espía podría intervenir.

In [56]:
def enviar_mensaje_unico(G, origen, destino, mensaje, espia_activo=True):
    """
    Simula el envío de un mensaje cifrado y firmado entre dos nodos de la red.

    Utiliza cifrado Vigenère para el mensaje y cifrado RSA para la clave simétrica.
    Opcionalmente, simula un ataque Man-in-the-Middle (MITM) donde un nodo espía
    intercepta el mensaje.

    Args:
        G (nx.Graph): El grafo que representa la red de dispositivos.
        origen (int): El índice del nodo de origen.
        destino (int): El índice del nodo de destino.
        mensaje (str): El mensaje a enviar.
        espia_activo (bool, optional): True para activar la simulación del espía, False para desactivarla.
                                       Por defecto es True.

    Returns:
        tuple: Una tupla que contiene el camino recorrido por el mensaje y el nodo espía (si está activo).
    """
    print(f"\n--- Simulando envío de mensaje ---")
    # Calcula el camino más corto entre origen y destino
    camino = nx.shortest_path(G, origen, destino)
    print(f"Camino más corto encontrado de {origen} a {destino}: {camino}")
    # Selecciona un nodo espía aleatorio en el camino (excluyendo origen y destino) si el espía está activo y el camino tiene al menos 3 nodos
    espia = random.choice(camino[1:-1]) if espia_activo and len(camino) > 2 else None
    if espia_activo and len(camino) > 2:
        print(f"Espía activo. Nodo espía seleccionado (si aplica): {espia}")
    else:
        print("Espía inactivo o camino demasiado corto para espía.")


    # Genera una clave simétrica aleatoria para el cifrado Vigenère
    clave_simetrica = ''.join(random.choice(ALFABETO) for _ in range(16))
    print(f"Clave simétrica generada para Vigenère: '{clave_simetrica}'")
    # Firma digitalmente el mensaje usando SHA256
    mensaje_firmado = firmar_mensaje(mensaje)
    # Cifra el mensaje firmado usando Vigenère con la clave simétrica
    mensaje_cifrado = vigenere_cifrar(mensaje_firmado, clave_simetrica)

    # Obtiene la clave pública del nodo destino
    public_key = G.nodes[destino]['rsa_public']
    print(f"\nObteniendo clave pública del nodo destino ({destino}).")
    # Crea un objeto de cifrado RSA con PKCS1_OAEP
    cipher_rsa = PKCS1_OAEP.new(public_key)
    print("Objeto de cifrado RSA creado.")
    # Cifra la clave simétrica usando la clave pública del destino
    clave_cifrada = cipher_rsa.encrypt(clave_simetrica.encode())
    print(f"Clave simétrica cifrada con RSA (destino).")


    # Imprime información sobre el envío del mensaje
    print(f"\n📤 Enviando mensaje de nodo {origen} ({G.nodes[origen]['ciudad']}) a nodo {destino} ({G.nodes[destino]['ciudad']}):")
    print(" - Camino:", [f"{n} ({G.nodes[n]['ciudad']})" for n in camino])
    print(" - Clave cifrada (hex):", binascii.hexlify(clave_cifrada).decode()[:50] + "..." if len(binascii.hexlify(clave_cifrada).decode()) > 50 else binascii.hexlify(clave_cifrada).decode()) # Mostrar solo una parte si es muy larga
    print(" - Mensaje cifrado:", mensaje_cifrado[:50] + "..." if len(mensaje_cifrado) > 50 else mensaje_cifrado) # Mostrar solo una parte si es muy larga

    # Simula la intercepción por el espía si está activo
    if espia is not None:
        print(f"\n⚠️ Nodo espía interceptó el mensaje en nodo {espia} ({G.nodes[espia]['ciudad']}):")
        print(" - Contenido interceptado:", mensaje_cifrado[:50] + "..." if len(mensaje_cifrado) > 50 else mensaje_cifrado) # Mostrar solo una parte si es muy larga
        # Simula que el espía intenta descifrar la clave simétrica (para demostración, asume que el espía tiene la clave privada del destino)
        try:
            print(" - Espía intentando descifrar la clave simétrica (simulación de ataque)...")
            spy_private_key = G.nodes[destino]['rsa_private'] # Espía obtiene la clave privada del destino (simulación)
            spy_clave_descifrada = PKCS1_OAEP.new(spy_private_key).decrypt(clave_cifrada).decode()
            print(" - Clave simétrica descifrada (por el espía):", spy_clave_descifrada)
            # Nota: En un escenario real, el espía no tendría la clave privada del destino.
        except Exception as e:
            print(f" - El espía no pudo descifrar la clave simétrica. Error: {e}")
            print(" - El ataque MITM simulado no tuvo éxito en descifrar la clave.")


    # Obtiene la clave privada del nodo destino
    print(f"\nNodo destino ({destino}) recibiendo mensaje.")
    private_key = G.nodes[destino]['rsa_private']
    print("Obteniendo clave privada del nodo destino.")
    # Descifra la clave simétrica usando la clave privada del destino
    print("Descifrando clave simétrica con clave privada del destino...")
    try:
        clave_descifrada = PKCS1_OAEP.new(private_key).decrypt(clave_cifrada).decode()
        print(f"Clave simétrica descifrada: '{clave_descifrada}'")
        # Descifra el mensaje cifrado usando Vigenère con la clave simétrica descifrada
        mensaje_descifrado = vigenere_descifrar(mensaje_cifrado, clave_descifrada)
        # Verifica la firma digital del mensaje descifrado
        valido, mensaje_final = verificar_firma(mensaje_descifrado)

        # Imprime información sobre el mensaje recibido
        print("\n📥 Mensaje recibido y procesado:")
        print(" - Mensaje descifrado (con firma):", mensaje_descifrado)
        print(" - Mensaje original extraído después de verificar firma:", mensaje_final)
        print(" - Integridad:", "✅ Válido" if valido else "❌ Alterado (Firma inválida)")

    except Exception as e:
        print(f"Error al descifrar la clave simétrica en el destino: {e}")
        valido = False
        mensaje_final = "Error en el descifrado."
        print(" - Integridad: ❌ Error en el descifrado.")


    # Devuelve el camino recorrido y el nodo espía (si existe)
    return camino, espia

print("\nFunción enviar_mensaje_unico definida.")


Función enviar_mensaje_unico definida.


(**mostrar_red_en_mapa_plotly**) Este código es la preparación inicial dentro de la función de visualización para obtener la
información necesaria del grafo (G) antes de empezar a dibujar el mapa.

In [57]:
# Nueva función para visualizar la red en un mapa interactivo usando Plotly
def mostrar_red_en_mapa_plotly(G, camino, origen, destino, espia=None):
    """
    Visualiza la red de dispositivos en un mapa interactivo usando Plotly.

    Args:
        G (nx.Graph): El grafo que representa la red de dispositivos.
        camino (list): La lista de nodos que forman el camino del mensaje.
        origen (int): El nodo de origen del mensaje.
        destino (int): El nodo de destino del mensaje.
        espia (int, optional): El nodo espía, si existe.
    """
    print("\nGenerando visualización de la red en un mapa interactivo usando Plotly...")
    # Obtiene las posiciones de los nodos
    pos = nx.get_node_attributes(G, 'pos')
    # Crea etiquetas para los nodos con su número y ciudad
    labels = {n: f"{n} ({G.nodes[n]['ciudad']})" for n in G.nodes}

    # Prepara los datos para las aristas del grafo (sin información de distancia)
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None]) # Use extend for cleaner code
        edge_y.extend([y0, y1, None]) # Use extend for cleaner code

    print(f"Datos preparados para {len(G.edges)} aristas.")

    # Crea el trazo para las aristas
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',  # No muestra información al pasar el ratón
        mode='lines')
    print("Trazo creado para las aristas de la red.")

    # Prepara los datos para los nodos del grafo
    node_x = []
    node_y = []
    node_text = []
    node_color = []
    node_size = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(labels[node])
        # Asigna colores y tamaños especiales a los nodos de origen, destino y espía
        if node == origen:
            node_color.append('orange')
            node_size.append(20)
            print(f"Nodo {node} identificado como Origen.")
        elif node == destino:
            node_color.append('red')
            node_size.append(20)
            print(f"Nodo {node} identificado como Destino.")
        elif espia is not None and node == espia:
            node_color.append('purple')
            node_size.append(20)
            print(f"Nodo {node} identificado como Espía.")
        else:
            node_color.append('lightblue')
            node_size.append(10)

    print(f"Datos preparados para {len(G.nodes)} nodos.")

    # Crea el trazo para los nodos
    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text',
        text=node_text,
        textposition="bottom center",
        hoverinfo='text',
        marker=dict(
            showscale=False,
            colorscale='YlGnBu',
            reversescale=True,
            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Conexiones de Nodos',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))
    print("Trazo creado para los nodos.")

    # Crear aristas para el camino recorrido por el mensaje (sin información de distancia)
    path_edge_x = []
    path_edge_y = []
    for i in range(len(camino) - 1):
        x0, y0 = pos[camino[i]]
        x1, y1 = pos[camino[i+1]]
        path_edge_x.extend([x0, x1, None]) # Use extend for cleaner code
        path_edge_y.extend([y0, y1, None]) # Use extend for cleaner code
    print(f"Datos preparados para el camino recorrido ({len(camino)-1} segmentos).")

    # Crea el trazo para el camino
    path_trace = go.Scatter(
        x=path_edge_x, y=path_edge_y,
        line=dict(width=2.5, color='green'),
        hoverinfo='none', # No muestra información al pasar el ratón
        mode='lines',
        name='Camino')
    print("Trazo creado para el camino del mensaje.")

    # Crear punto animado para el mensaje que se mueve a lo largo del camino
    message_point = go.Scatter(
        x=[pos[camino[0]][0]], y=[pos[camino[0]][1]],
        mode='markers',
        marker=dict(size=10, color='darkgreen'),
        name='Mensaje')
    print("Trazo creado para el punto animado del mensaje.")


    # Crea la figura de Plotly con los trazos
    fig = go.Figure(data=[edge_trace, node_trace, path_trace, message_point],
                 layout=go.Layout(
                    title='Red de Dispositivos y Camino del Mensaje', # Título actualizado
                    titlefont_size=16,
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[
                     ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                   )
    print("Figura de Plotly creada.")

    # Crear frames para la animación
    frames = []
    frames_per_edge = 10 # Esto debe coincidir con la velocidad de animación deseada
    total_frames = (len(camino) - 1) * frames_per_edge if len(camino) > 1 else frames_per_edge
    print(f"Creando {total_frames} frames para la animación del mensaje.")
    for i in range(total_frames):
        edge_index = min(i // frames_per_edge, len(camino) - 2) # Ensure index is within bounds
        start_node = camino[edge_index]
        end_node = camino[edge_index + 1]
        start_pos = pos[start_node]
        end_pos = pos[end_node]
        t = (i % frames_per_edge) / frames_per_edge
        x = start_pos[0] + (end_pos[0] - start_pos[0]) * t
        y = start_pos[1] + (end_pos[1] - start_pos[1]) * t
        frames.append(go.Frame(data=[go.Scatter(x=[x], y=[y], mode='markers', marker=dict(size=10, color='darkgreen'))], name=str(i))) # Added mode and marker

    # Asigna los frames a la figura
    fig.frames = frames
    print("Frames de animación asignados a la figura.")

    # Agregar botón de reproducción y slider para controlar la animación
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                buttons=[dict(label="Reproducir",
                              method="animate",
                              args=[None, {"frame": {"duration": 100, "redraw": True},
                                          "fromcurrent": True, "transition": {"duration": 0}}],
                              args2=[None, {"frame": {"duration": 100, "redraw": True},
                                           "mode": "immediate", "transition": {"duration": 0}}])],
                pad={"r": 10, "t": 87},
                showactive=False,
                x=0.1,
                xanchor="right",
                y=0,
                yanchor="top"
            )
        ],
        sliders=[dict(steps=[dict(args=[[f.name]], # Corrected args for slider steps
                                  label=k,
                                  method="animate") for k, f in enumerate(fig.frames)],
                      transition=dict(duration=0),
                      x=0.1,
                      xanchor="left",
                      y=0,
                      yanchor="top")]
    )
    print("Botón de reproducción y slider agregados a la figura.")


    # Muestra la figura interactiva
    print("Mostrando figura interactiva...")
    fig.show()
    print("Visualización de la red mostrada.")

print("\nFunción mostrar_red_en_mapa_plotly definida.")


Función mostrar_red_en_mapa_plotly definida.


Este fragmento de código corresponde a la función (**configurar_y_enviar_mensaje_unico**).

Su propósito es interactuar con el usuario para obtener la información necesaria para
enviar un mensaje a través de la red simulada.

In [58]:
def configurar_y_enviar_mensaje_unico(G):
    """
    Configura y envía un solo mensaje, solicitando origen, destino y mensaje al usuario.
    """
    print("\n--- Configurando y enviando un mensaje único ---")
    # Muestra los nodos disponibles y sus ciudades
    print(f"\nNodos disponibles: {[(n, G.nodes[n]['ciudad']) for n in G.nodes]}")

    # --- Entrada para el Mensaje ---
    print("\n--- Configuración del Mensaje ---")
    while True:
        try:
            # Solicita al usuario el nodo origen
            origen_input = input("Nodo origen: ")
            origen = int(origen_input)
            # Verifica si el nodo origen es válido
            if origen not in G.nodes:
                print("Nodo de origen no válido. Por favor, ingresa un nodo de la lista disponible.")
                continue
            # Solicita al usuario el nodo destino
            destino_input = input("Nodo destino: ")
            destino = int(destino_input)
            # Verifica si el nodo destino es válido
            if destino not in G.nodes:
                print("Nodo de destino no válido. Por favor, ingresa un nodo de la lista disponible.")
                continue
            # Verifica si el nodo origen y destino son diferentes
            if origen == destino:
                 print("El nodo origen y destino no pueden ser el mismo.")
                 continue
            # Verifica si existe un camino entre los nodos seleccionados
            if not nx.has_path(G, origen, destino):
                 print(f"No existe un camino entre el nodo {origen} y el nodo {destino}.")
                 continue
            # Sale del bucle si los nodos origen y destino son válidos y diferentes y hay un camino
            break
        except ValueError:
            # Maneja el error si la entrada no es un número entero
            print("Entrada inválida. Por favor, ingresa un número entero.")
        except Exception as e:
            # Maneja cualquier otro error
            print(f"Ocurrió un error al obtener los nodos: {e}")

    # Solicita al usuario el mensaje a enviar
    mensaje = input("Mensaje a enviar: ")

    # Solicita al usuario si desea activar el espía
    while True:
        activar_espia_input = input("¿Activar espía (MITM)? (s/n): ").lower()
        # Verifica si la entrada es válida ('s' o 'n')
        if activar_espia_input in ['s', 'n']:
            activar_espia = activar_espia_input == 's' # Asigna True if the input is 's', False if it is 'n'
            break # Sale del bucle después de una entrada válida
        else:
            print("Entrada inválida. Por favor, ingresa 's' o 'n'.")

    print(f"\nConfiguración completa: Origen={origen}, Destino={destino}, Mensaje='{mensaje}', Espía activo={activar_espia}")

    # Llama a la función para enviar el mensaje
    camino, espia = enviar_mensaje_unico(G, origen, destino, mensaje, activar_espia)

    # Usar la nueva función de visualización de Plotly
    mostrar_red_en_mapa_plotly(G, camino, origen, destino, espia)
    print("\nSimulación de envío de mensaje único finalizada.")

print("\nFunción configurar_y_enviar_mensaje_unico definida.")


Función configurar_y_enviar_mensaje_unico definida.


Finalmente, llegamos a la explicación de la función **main()**.

Esta es la función principal de tu
script y actúa como el punto de entrada cuando ejecutas el código. Se encarga de la configuración inicial y de presentar el menú interactivo al usuario.

In [60]:
def main():
    """
    Función principal para ejecutar el simulador de red cifrada con menú interactivo.
    """
    print("\n=== Visualización de Comunicación Segura con Cifrado RSA + Vigenère en una Red Geográfica ===")
    # Obtiene el número de ciudades disponibles
    num_ciudades = len(CIUDADES_COORDS)
    print(f"Número de ciudades disponibles para nodos: {num_ciudades}")

    G = None # Inicializar G fuera del bucle del menú (el grafo se crea una vez)

    while True:
        # Solo crear el grafo si aún no existe (se crea al inicio de la ejecución)
        if G is None:
            print("\n--- Creando la Red de Dispositivos ---")
            while True:
                try:
                    # Solicita al usuario la cantidad de nodos para la red
                    n_input = input(f"¿Cuántos nodos quieres en la red? (mínimo 3, máximo {num_ciudades}): ")
                    n = int(n_input)
                    # Verifica si la cantidad de nodos está dentro del rango permitido
                    if n >= 3 and n <= num_ciudades:
                        G = crear_red_dispositivos(n) # Crea la red de dispositivos
                        print(f"\nRed de {n} nodos creada exitosamente.")
                        break # Sale del bucle si la cantidad de nodos es válida
                    else:
                        print(f"Cantidad de nodos no disponible. Por favor, ingresa un número entre 3 y {num_ciudades}.")
                except ValueError:
                    # Maneja el error si la entrada no es un número entero
                    print("Entrada inválida. Por favor, ingresa un número entero.")
                except Exception as e:
                    print(f"Ocurrió un error al crear la red: {e}")


        print("\n--- Menú Principal ---")
        print("1. Enviar un mensaje")
        print("3. Salir")
        # Solicita al usuario que seleccione una opción del menú
        opcion_menu_principal = input("Selecciona una opción: ")
        print(f"Opción seleccionada: {opcion_menu_principal}")

        # Procesa la opción seleccionada por el usuario
        if opcion_menu_principal == '1':
            configurar_y_enviar_mensaje_unico(G) # Llama a la función para configurar y enviar un mensaje único
        elif opcion_menu_principal == '3':
            print("Saliendo del simulador. ¡Hasta luego!") # Mensaje de despedida
            break # Sale del bucle principal para terminar el programa
        else:
            print("Opción no válida. Por favor, selecciona 1 o 3.") # Mensaje para opción inválida


if __name__ == "__main__":
    main() # Ejecuta la función principal if the script is executed directly


=== Visualización de Comunicación Segura con Cifrado RSA + Vigenère en una Red Geográfica ===
Número de ciudades disponibles para nodos: 27

--- Creando la Red de Dispositivos ---
¿Cuántos nodos quieres en la red? (mínimo 3, máximo 27): 15

Creando una red aleatoria de 15 dispositivos.
Ciudades disponibles mezcladas para asignación: ['Florencia', 'Tunja', 'Cúcuta', 'Apartadó', 'Manizales']...
Grafo inicial creado con 15 nodos y 45 aristas.
Grafo conexo creado exitosamente.
Asignando claves RSA y ubicación a cada nodo...
Claves RSA y ubicaciones asignadas a todos los nodos.

Red de 15 nodos creada exitosamente.

--- Menú Principal ---
1. Enviar un mensaje
3. Salir
Selecciona una opción: 1
Opción seleccionada: 1

--- Configurando y enviando un mensaje único ---

Nodos disponibles: [(0, 'Florencia'), (1, 'Tunja'), (2, 'Cúcuta'), (3, 'Apartadó'), (4, 'Manizales'), (5, 'Villavicencio'), (6, 'Riohacha'), (7, 'Valledupar'), (8, 'Bucaramanga'), (9, 'Leticia'), (10, 'Bogotá'), (11, 'Cali'), (1

Visualización de la red mostrada.

Simulación de envío de mensaje único finalizada.

--- Menú Principal ---
1. Enviar un mensaje
3. Salir


KeyboardInterrupt: Interrupted by user